In [2]:
!pip install -qU langchain-community langgraph langchain-anthropic tavily-python langgraph-checkpoint-sqlite
!pip install -qU "langchain[openai]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 69.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.9/154.9 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.1/286.1 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.5/438.5 kB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.6/151.6 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.4/63.4 kB 4.8 MB/s eta 0:00:00


In [3]:
from google.colab import userdata
import os

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
os.environ["LANGSMITH_API_KEY"] = userdata.get("LANGSMITH_API_KEY")
os.environ["LANGSMITH_TRACING"] = userdata.get("LANGSMITH_TRACING")
os.environ["LANGSMITH_PROJECT"] = userdata.get("LANGSMITH_PROJECT")
os.environ["TAVILY_API_KEY"] = userdata.get("TAVILY_API_KEY")

In [22]:
from langchain_openai import ChatOpenAI
import os

model = ChatOpenAI(
    model="mistralai/mistral-small-3.1-24b-instruct:free",
    base_url="https://openrouter.ai/api/v1",
    api_key=os.getenv("OPENAI_API_KEY"),
    temperature=0.3
)

In [9]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in Indore")
print(search_results)
tools = [search]

[{'title': 'Indore Weather Forecast 31 May 2025 - Times of India', 'url': 'https://timesofindia.indiatimes.com/weather/indore-weather-forecast-today/452007', 'content': "Today's Weather in Indore: In Indore today, the weather is expected to be Haze with a maximum temperature of 34°C and a minimum of 24°C. Sunrise in Indore", 'score': 0.9477354}, {'title': 'Weather for Indore, Madhya Pradesh, India - Time and Date', 'url': 'https://www.timeanddate.com/weather/india/indore', 'content': 'Latest Report: May 31, 2025 at 5:30 am. Visibility: 2 mi. Pressure: 29.58 "Hg (27.77 "Hg at 554m altitude). Humidity: 83%. Dew Point: 73 °F. Location of Indore', 'score': 0.93206495}]


In [7]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="hi!")])
response.content

'Hello! How are you today? Is there something I can help you with or would you like to chat?'

In [24]:
model_with_tools = model.bind_tools(tools)

In [11]:
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Hi! How can I help you today?
ToolCalls: []


In [25]:
response = model_with_tools.invoke([HumanMessage(content="What's the weather in Jabalpur, Madhya Pradesh?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Jabalpur, Madhya Pradesh'}, 'id': 'FtcNIsoeJ', 'type': 'tool_call'}]


In [26]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

In [27]:
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]

[HumanMessage(content='hi!', additional_kwargs={}, response_metadata={}, id='086f0b62-dd9d-4bbc-ac9f-ba0818bcb5bd'),
 AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 104, 'total_tokens': 114, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'mistralai/mistral-small-3.1-24b-instruct:free', 'system_fingerprint': None, 'id': 'gen-1748707312-51tvvHPKxMk8twVHGmyi', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--a1599df6-9f04-4f30-8aad-1c3caaeac413-0', usage_metadata={'input_tokens': 104, 'output_tokens': 10, 'total_tokens': 114, 'input_token_details': {}, 'output_token_details': {}})]

In [28]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
)
response["messages"]

[HumanMessage(content='whats the weather in sf?', additional_kwargs={}, response_metadata={}, id='2e01127a-7dc5-448c-848c-d49902342565'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '3VqNbL5lY', 'function': {'arguments': '{"query": "weather in sf"}', 'name': 'tavily_search_results_json'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 109, 'total_tokens': 135, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'mistralai/mistral-small-3.1-24b-instruct:free', 'system_fingerprint': None, 'id': 'gen-1748707840-1nS71dzuVwATIkNZjkv0', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--f1e5586c-9999-4dbd-93b5-24c6aa6a8544-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in sf'}, 'id': '3VqNbL5lY', 'type': 'tool_call'}], usage_metadata={'input_tokens': 109, 'output_tokens': 26, 'total_tokens': 135, 'i

In [29]:
for step in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in Jabalpur?")]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

whats the weather in Jabalpur?
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (vkLXduhFm)
 Call ID: vkLXduhFm
  Args:
    query: weather in Jabalpur
================================= Tool Message =================================
Name: tavily_search_results_json

[{"title": "Jabalpur - Local Weather Report and Forecast", "url": "https://city.imd.gov.in/citywx/city_weather_test_try_warnings.php?id=42675", "content": "Local Weather Report and Forecast For: Jabalpur Dated :May 31, 2025. Jabalpur. Past 24 Hours Weather Data. Maximum Temp(oC) (Recorded. on 31/05/25), 37.2.", "score": 0.9511106}, {"title": "Jabalpur Madhya, PRADESH | Weather Forecasts Now, Live Radar ...", "url": "https://www.weatherbug.com/weather-forecast/now/jabalpur-madhya-pradesh-in", "content": "Now · Hourly · 10 Day. Today's Weather - Jabalpur, IND. May 31, 2025 7

In [30]:
for step, metadata in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in sf?")]},
    stream_mode="messages",
):
    if metadata["langgraph_node"] == "agent" and (text := step.text()):
        print(text, end="|")

The| weather| in| San| Francisco| on| May| |3|1|,| |2|0|2|5|,| is| expected| to| be| cloud|y| with| over|cast| skies|.| The| temperature| will| range| from| |7|0|°|F| during| the| day| to| |5|5|°|F| at| night|,| with| |0|%| chance| of| precipitation| [|REF|]|p|um|D|Hi|OS|3|[|/|REF|].|

In [31]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [32]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

In [33]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im Pranshu!")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Hello Pranshu! Nice to meet you. How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 108, 'total_tokens': 126, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'mistralai/mistral-small-3.1-24b-instruct:free', 'system_fingerprint': None, 'id': 'gen-1748713516-qDpspdFwfe5ym66dbY4N', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--2da29ac8-dc0a-489b-bc49-cb01956decf1-0', usage_metadata={'input_tokens': 108, 'output_tokens': 18, 'total_tokens': 126, 'input_token_details': {}, 'output_token_details': {}})]}}
----


In [35]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Based on our previous conversation, your name is Pranshu. If you have any questions or need assistance with something else, feel free to ask!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 168, 'total_tokens': 199, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'mistralai/mistral-small-3.1-24b-instruct:free', 'system_fingerprint': None, 'id': 'gen-1748713612-8LE9xC9N17GwCu6cnoDG', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--2e31069e-cdb0-4e11-8809-a24786298c63-0', usage_metadata={'input_tokens': 168, 'output_tokens': 31, 'total_tokens': 199, 'input_token_details': {}, 'output_token_details': {}})]}}
----


In [36]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="I live in Indore")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Great! Indore is a vibrant city in India. How can I help you today, Pranshu? If you have any questions or need information about Indore or anything else, feel free to ask!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 206, 'total_tokens': 249, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'mistralai/mistral-small-3.1-24b-instruct:free', 'system_fingerprint': None, 'id': 'gen-1748713632-aHCz1btf76iBZL0XCXQt', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--df5cf93d-ec77-41a9-933e-555090a14775-0', usage_metadata={'input_tokens': 206, 'output_tokens': 43, 'total_tokens': 249, 'input_token_details': {}, 'output_token_details': {}})]}}
----


In [38]:
for step, metadata in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather where I live?")]},
    stream_mode="messages",
    config=config
):
    if metadata["langgraph_node"] == "agent" and (text := step.text()):
        print(text, end="")

The weather in Indore today is expected to be hazy with a maximum temperature of 34°C and a minimum of 24°C [REF]0[/REF]. For the upcoming days, here's a brief forecast:

- **Saturday, May 31, 2025:** Cloudy with a temperature of 35°C [REF]1[/REF].
- **Sunday, June 1, 2025:** Cloudy with a temperature of 36°C [REF]1[/REF].
- **Monday, June 2, 2025:** Cloudy with a temperature of 36°C [REF]1[/REF].
- **Tuesday, June 3, 2025:** Rainy with a temperature of 34°C [REF]1[/REF].
- **Wednesday, June 4, 2025:** Clear skies [REF]1[/REF].